[Reference](https://towardsdatascience.com/build-your-own-movie-recommender-system-using-bert4rec-92e4e34938c5)

In [2]:
pip install pytorch-lightning

     |████████████████████████████████| 925 kB 7.3 MB/s 
     |████████████████████████████████| 282 kB 64.8 MB/s 
     |████████████████████████████████| 636 kB 55.8 MB/s 
     |████████████████████████████████| 125 kB 58.1 MB/s 
     |████████████████████████████████| 829 kB 59.1 MB/s 
     |████████████████████████████████| 1.3 MB 54.5 MB/s 
     |████████████████████████████████| 160 kB 50.3 MB/s 
     |████████████████████████████████| 271 kB 59.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=088b485eb3c68e9b7a8b9f4132cb77269a2b46eb398196dd6b47db53f467695b
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [4]:
!git clone https://github.com/CVxTz/recommender_transformer

Cloning into 'recommender_transformer'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 113 (delta 51), reused 69 (delta 23), pack-reused 0
Receiving objects: 100% (113/113), 30.93 KiB | 7.73 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [9]:
!pwd

/content


In [10]:
import sys
sys.path
sys.path.append('/content/recommender_transformer')

In [11]:
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

from recommender.models import Recommender
from recommender.data_processing import get_context, pad_list, map_column, MASK, PAD

In [16]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2021-10-10 08:45:07--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  47.4MB/s    in 5.7s    

2021-10-10 08:45:13 (43.5 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [17]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [19]:
!ls

ml-25m	ml-25m.zip  recommender_transformer  sample_data


In [34]:
data_csv_path = "ml-25m/ratings.csv"
movies_path = "ml-25m/movies.csv"

model_path = "recommender_transformer/recommender_models/recommender.ckpt"

In [36]:
data = pd.read_csv(data_csv_path)
movies = pd.read_csv(movies_path)

In [37]:
data.sort_values(by="timestamp", inplace=True)

In [38]:
data, mapping, inverse_mapping = map_column(data, col_name="movieId")
grp_by_train = data.groupby(by="userId")

In [39]:
random.sample(list(grp_by_train.groups), k=10)

[55753, 58075, 57409, 86946, 158919, 38293, 76926, 59410, 82202, 90564]

In [46]:
model = Recommender(
        vocab_size=len(mapping) + 2,
        lr=1e-4,
        dropout=0.3,
    )
model.eval()
model.load_state_dict(torch.load(model_path)["state_dict"])

In [41]:
movie_to_idx = {a: mapping[b] for a, b in zip(movies.title.tolist(), movies.movieId.tolist()) if b in mapping}
idx_to_movie = {v: k for k, v in movie_to_idx.items()}

In [42]:
def predict(list_movies, model, movie_to_idx, idx_to_movie):
    
    ids = [PAD] * (120 - len(list_movies) - 1) + [movie_to_idx[a] for a in list_movies] + [MASK]
    
    src = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    
    with torch.no_grad():
        prediction = model(src)
    
    masked_pred = prediction[0, -1].numpy()
    
    sorted_predicted_ids = np.argsort(masked_pred).tolist()[::-1]
    
    sorted_predicted_ids = [a for a in sorted_predicted_ids if a not in ids]
    
    return [idx_to_movie[a] for a in sorted_predicted_ids[:30] if a in idx_to_movie]

# Senario 1: Adventure/Fantasy


In [43]:
list_movies = ["Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
               "Harry Potter and the Chamber of Secrets (2002)",
               "Harry Potter and the Prisoner of Azkaban (2004)",
               "Harry Potter and the Goblet of Fire (2005)"]

top_movie = predict(list_movies, model, movie_to_idx, idx_to_movie)
top_movie

# Senario 2: Action/Adventure


In [44]:
list_movies = ["Black Panther (2017)",
               "Avengers, The (2012)",
               "Avengers: Infinity War - Part I (2018)",
               "Logan (2017)",
               "Spider-Man (2002)",
               "Spider-Man 3 (2007)",
               "Spider-Man: Far from Home (2019)"]

top_movie = predict(list_movies, model, movie_to_idx, idx_to_movie)
top_movie

# Senario 3: Comedy

In [45]:
list_movies = ["Zootopia (2016)",
               "Toy Story 3 (2010)",
               "Toy Story 4 (2019)",
               "Finding Nemo (2003)",
               "Ratatouille (2007)",
               "The Lego Movie (2014)",
               "Ghostbusters (a.k.a. Ghost Busters) (1984)",
               "Ace Ventura: When Nature Calls (1995)"]
top_movie = predict(list_movies, model, movie_to_idx, idx_to_movie)
top_movie